# Variational Auto Encoder
The logic for this has been taken from the original paper and Lecture 13 of CS231n

## Model

In [6]:
from scipy.special import dtype


class VariationalAutoencoder(nn.Module):

    def __init__(self, input_dim, hidden_dim = 128*128, latent_dim = 32*32):
        super(VariationalAutoencoder, self).__init__()

        #Encoding
        self.img2hid = nn.Linear(input_dim, hidden_dim)
        self.hid2mu = nn.Linear(hidden_dim, latent_dim)
        self.hid2var = nn.Linear(hidden_dim, latent_dim)

        #Decoding
        self.z2hid = nn.Linear(latent_dim, hidden_dim)
        self.hid2img = nn.Linear(hidden_dim, input_dim)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):

        h = self.relu(self.img2hid(x))
        mu = self.hid2mu(h)
        sigma = self.hid2var(h)

        return mu, sigma

    def decode(self, z):

        h = self.relu(self.z2hid(z))

        return self.sigmoid(self.hid2img(h))

    def forward(self, x):

        mu, sigma = self.encode(x)
        epsilon = torch.randn_like(sigma)
        z_reparametarized = mu + epsilon * sigma
        reconstructed = self.decode(z_reparametarized)

        return reconstructed, mu, torch.log(sigma)

In [259]:
import torch
from torch import nn

class LSTM(nn.Module):

    def __init__(self):
        super(LSTM, self).__init__()

        self.lstm = nn.LSTM(input_size=5, hidden_size=128, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(in_features=128, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):

        _, (h, c) = self.lstm(x)
        lh = h[-1]

        out = self.fc1(lh).squeeze(-1)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.fc2(out)
        out = self.sigmoid(out)

        return out

## Data Generation and Augmentation

First part is AI generated.\
Dataset is generated on ~10 min clips of CSGO2 recorded at 720p @60 fps with .h264 encoding it was not running on AV1 encoding for me.

In [ ]:
import torch
import numpy as np
import cv2

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.to('cuda')
model.eval()

In [ ]:
def extract_features_from_csgo_video(video_path, output_path, max_frames=37201):   #This whole part is AI generated although logic was mine
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file.")
        return

    features = []
    ret, prev_frame = cap.read()

    if not ret:
        print("Error reading first frame.")
        return

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    prev_brightness = prev_gray.mean()

    frame_count = 0
    h, w = prev_gray.shape
    center_x, center_y = w / 2, h / 2


    while True:

        ret, frame = cap.read()
        if not ret or frame_count >= max_frames:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        brightness = gray.mean()

        # Detect firing event (simple brightness spike)
        is_firing = 1 if (brightness - prev_brightness) > 15 else 0
        prev_brightness = brightness

        # Optical flow for dx, dy (approximates mouse movement)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray,None, 0.5, 3, 15, 3, 5, 1.2, 0)
        dx = flow[center_y, center_x, 0]
        dy = flow[center_y, center_x, 1]
        prev_gray = gray

        # Dummy x, y target position: set to 0 (no detection)
        # You can add YOLOv8 to track enemies in future
        with torch.no_grad():
            results = model(frame)
        detections = results.xyxy[0]

        h, w, _ = frame.shape
        cx, cy = w // 2, h // 2  # crosshair center

        closest_dist = float('inf')
        x_pos, y_pos = 0.0, 0.0

        for *xyxy, conf, cls in detections:
            if int(cls) != 0:  # class 0 is 'person' in COCO
                continue
            x1, y1, x2, y2 = map(int, xyxy)
            px, py = (x1 + x2) / 2, (y1 + y2) / 2  # enemy center

            dx, dy = px - cx, py - cy
            dist = (dx**2 + dy**2)**0.5

            if dist < closest_dist:
                closest_dist = dist
                x_pos = dx
                y_pos = dy


        features.append([x_pos, y_pos, dx, dy, is_firing])
        frame_count += 1

    cap.release()

    features = np.array(features)
    print(f"Extracted {features.shape[0]} frames with {features.shape[1]} features each.")

    # Save as PyTorch tensor
    tensor = torch.tensor(features, dtype=torch.float32)
    torch.save(tensor, output_path)
    print(f"Saved features to: {output_path}")


In [19]:
import os

search_in = os.listdir("C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/dataset/")

In [142]:
path = "C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/dataset/"
dataset = []
dataset1 = []

for file in search_in:
    if(file.startswith("features_ai")):
        dataset.append(torch.load(path + file))

for file in search_in:
    if(file.startswith("features")):
        dataset1.append(torch.load(path + file))

In [145]:
min_len = min([len(i) for i in dataset])
min_len1 = min([len(i) for i in dataset1])
min_len = min(min_len, min_len1)

In [147]:
for i, j in enumerate(dataset):

    j = j[:min_len]
    print(j)
    if i == 0:
        dataset_tensor_x_ai = torch.tensor(j, dtype=torch.float32)
    else:
        dataset_tensor_x_ai = torch.cat([dataset_tensor_x_ai, torch.tensor(j, dtype=torch.float32)], dim=0)

for i, j in enumerate(dataset1):

    j = j[:min_len]
    print(j)
    if i == 0:
        dataset_tensor_x = torch.tensor(j, dtype=torch.float32)
    else:
        dataset_tensor_x = torch.cat([dataset_tensor_x, torch.tensor(j, dtype=torch.float32)], dim=0)


tensor([[   5.,   18.,    5.,   18.,    0.],
        [   7.,   20.,  -18., -313.,    0.],
        [   7.,   16.,  -18., -313.,    0.],
        ...,
        [  36.,   19.,   36.,   19.,    0.],
        [  39.,   16.,  -19., -312.,    0.],
        [  30.,   20.,   30.,   20.,    0.]])
tensor([[ 0.0000,  0.0000, -4.0788,  6.6022,  0.0000],
        [ 0.0000,  0.0000, -1.1287, -0.8875,  0.0000],
        [ 0.0000,  0.0000, -2.2969, -0.9227,  0.0000],
        ...,
        [ 0.0000,  0.0000,  9.0995,  5.0711,  0.0000],
        [ 0.0000,  0.0000, -2.0096, -1.5862,  0.0000],
        [ 0.0000,  0.0000, -7.3035, -1.6283,  0.0000]])
tensor([[  20.,   10.,  137., -313.,    0.],
        [  22.,   12.,  137., -313.,    0.],
        [  20.,   17.,  137., -313.,    0.],
        ...,
        [  46.,   38.,  175., -312.,    0.],
        [  41.,   39.,  -17., -310.,    0.],
        [  32.,   34.,   96., -312.,    0.]])
tensor([[-1.0000e+00,  5.7000e+01,  1.7400e+02, -3.1100e+02,  0.0000e+00],
        [ 0.0

C:\Users\VIDIT\AppData\Local\Temp\ipykernel_25968\545388814.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset_tensor_x_ai = torch.tensor(j, dtype=torch.float32)
C:\Users\VIDIT\AppData\Local\Temp\ipykernel_25968\545388814.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset_tensor_x_ai = torch.cat([dataset_tensor_x_ai, torch.tensor(j, dtype=torch.float32)], dim=0)
C:\Users\VIDIT\AppData\Local\Temp\ipykernel_25968\545388814.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset_tensor_x = torch.tensor(j, dtype=torch.f

In [208]:
dataset_tensor_y_ai = torch.ones((16065,1))
dataset_tensor_y = torch.zeros([35700, 1])
print(dataset_tensor_x.shape, dataset_tensor_x_ai.shape)
print(dataset_tensor_y.shape, dataset_tensor_y_ai.shape)

torch.Size([35700, 5]) torch.Size([16065, 5])
torch.Size([35700, 1]) torch.Size([16065, 1])


In [205]:
input_vector = dataset_tensor_x_ai.unsqueeze(0)
print(input_vector.shape)

torch.Size([1, 16065, 5])


In [237]:
labels_ai = torch.ones((16065,1))


## Training

In [267]:
model = LSTM()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs =30

In [268]:
model = model.to('cuda')
model.train()

LSTM(
  (lstm): LSTM(5, 128, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=128, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [274]:
for epoch in range(epochs):

    epoch_loss = 0
    temp = []
    for i in range(59):
        idx = 600*i


        trimmed_dataset = dataset_tensor_x_ai[idx:idx+600]
        M, N =trimmed_dataset.shape
        if M != 0:
            temp.append(i)
            trimmed_dataset = trimmed_dataset.unsqueeze(0).to('cuda')
            trimmed_labels = torch.tensor([1], dtype=torch.float32).to('cuda')

            prediction = model(trimmed_dataset)[0]
            loss = criterion(prediction, trimmed_labels)
            epoch_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


    print(f"Epoch {epoch + 1}, loss: {epoch_loss}")
    print(temp)

torch.save(model.state_dict(), 'C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/weights.pth')

Epoch 1, loss: 1.7881407217146261e-06
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
Epoch 2, loss: 3.337863404340169e-06
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
Epoch 3, loss: 1.0252024566170803e-05
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
Epoch 4, loss: 1.6927767816810046e-05
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
Epoch 5, loss: 2.014652315551757e-05
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
Epoch 6, loss: 8.106239576477492e-06
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
Epoch 7, loss: 6.43730700744527e-06
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
Epoch 8, loss: 1.5497210625881053e-06
[0, 1, 2

In [264]:
torch.load('C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/weights.pth')

OrderedDict([('lstm.weight_ih_l0',
              tensor([[ 0.06258, -0.04522,  0.02309,  0.02810,  0.04117],
                      [ 0.05864,  0.05619, -0.00734,  0.05666,  0.04502],
                      [ 0.06577, -0.06241, -0.02655,  0.04106,  0.04416],
                      ...,
                      [ 0.06432,  0.00583,  0.06966, -0.08196,  0.02658],
                      [ 0.03228, -0.00725, -0.00028, -0.05288,  0.05377],
                      [ 0.04805,  0.02259, -0.04385,  0.02345, -0.07505]], device='cuda:0')),
             ('lstm.weight_hh_l0',
              tensor([[ 0.05371, -0.07605,  0.01565,  ...,  0.02333, -0.05438, -0.09819],
                      [ 0.09534,  0.03303, -0.06903,  ..., -0.00766,  0.05520, -0.06611],
                      [-0.09008,  0.04567,  0.08571,  ..., -0.03650,  0.04828,  0.04461],
                      ...,
                      [ 0.04921,  0.04412, -0.06106,  ..., -0.03956, -0.01812, -0.03781],
                      [ 0.03549, -0.06308,  0.01106,